In [1]:
import pandas as pd
import numpy as np

In [2]:
from google.colab import drive
drive.mount("/content/drive")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [3]:
train = pd.read_csv('/content/drive/My Drive/recsys-hw2/train.csv')
songs = pd.read_csv('/content/drive/My Drive/recsys-hw2/songs.csv')

In [4]:
train.head(5)

,msno,song_id,source_system_tab,source_screen_name,source_type,target
0,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,BBzumQNXUHKdEBOB7mAJuzok+IJA1c2Ryg/yzTF6tik=,explore,Explore,online-playlist,1
1,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,bhp/MpSNoqoxOIB+/l8WPqu6jldth4DIpCm3ayXnJqM=,my library,Local playlist more,local-playlist,1
2,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,JNWfrrC7zNN7BdMpsISKa4Mw+xVJYNnxXh3/Epw7QgY=,my library,Local playlist more,local-playlist,1
3,Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,2A87tzfnJTSWqD7gIZHisolhe4DMdzkbd6LzO1KHjNs=,my library,Local playlist more,local-playlist,1
4,FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,3qm6XTZ6MOCU11x8FIVbAGH5l5uMkT3/ZalWG1oo2Gc=,explore,Explore,online-playlist,1


In [5]:
songs.head(5)

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0


In [6]:
train.drop(['source_system_tab', 'source_screen_name', 'source_type'], axis=1, inplace=True)

train = train[train['target'] == 1]

target = train['target']

train.drop(['target'], axis=1, inplace=True)

In [7]:
temp = train.groupby(['msno'])['song_id'].apply(list)
temp = temp.to_frame().reset_index()

temp.columns = ['msno', 'song_id_history']

histories = list(temp['song_id_history'])

print(len(histories[0]))

293


Для каждого пользователя вытащили историю его лайков, обучим на них skipgram. Взял не очень большие размер эмбеддинга (64) и окно (5), чтобы училось чуть-чуть быстрее.

In [8]:
%%time

from gensim.models import Word2Vec

model = Word2Vec(window=5, sg=1, hs=0, size=64, negative=15, alpha=0.03, min_alpha=0.0007)
model.build_vocab(histories, progress_per=100)

model.train(histories, total_examples=model.corpus_count, epochs=10, report_delay=1)

CPU times: user 15min 39s, sys: 777 ms, total: 15min 39s
Wall time: 5min 25s


In [9]:
print(model[model.wv.vocab].shape)

(55816, 64)


/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [10]:
print(model[model.wv.vocab][0])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


[-0.21026215 -0.21907547 -0.24418934 -0.1990588  -0.883448    0.5492157
  0.460374   -0.2067067   0.1746037  -0.11719668  0.97223    -0.02667366
 -0.0433716  -0.08816152 -0.17095849 -0.23399873  0.37236282  0.31661746
  0.0831437   0.10915186  0.14463888  0.44264695 -0.6783294  -0.06961908
  0.5325378  -0.0734586  -0.2005061   0.8451047  -0.40111002 -0.25154147
 -0.18716572  1.1008812   0.03107803 -0.18050763  0.25047576  0.4464444
  0.32369924  0.02690399  0.0949702   0.2885624  -0.43931878  0.3938591
  0.33923978 -0.75368005  0.0539429   0.1678574  -0.98702276 -0.4033384
 -0.04532693 -0.51851124  0.1601011  -0.16902839 -0.28049377 -0.23418777
  0.06987635 -0.02598332  0.28869376 -0.04344971 -0.4473675   0.2624831
 -0.5233734   0.3073005  -0.53402865  0.28218353]


In [11]:
model.init_sims(replace=True)

In [12]:
def get_similars(initial_song_id, k=20):
    similars = model.similar_by_vector(model[initial_song_id], topn=k + 1)
    res = []
    
    for song_id, similarity in similars:
        row = songs[songs['song_id'] == song_id].to_dict('records')[0]
        row['similarity'] = similarity
        res.append(row)
        
    return pd.DataFrame(res)

In [13]:
initial_song_id = list(model.wv.vocab.items())[4000][0]
get_similars(initial_song_id)

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  
/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:2: DeprecationWarning: Call to deprecated `similar_by_vector` (Method will be removed in 4.0.0, use self.wv.similar_by_vector() instead).
  
/usr/local/lib/python3.6/dist-packages/gensim/matutils.py:737: FutureWarning: Conversion of the second argument of issubdtype from `int` to `np.signedinteger` is deprecated. In future, it will be treated as `np.int64 == np.dtype(int).type`.
  if np.issubdtype(vec.dtype, np.int):


,song_id,song_length,genre_ids,artist_name,composer,lyricist,language,similarity
0,7HDcHFAVdw76mL4Z5hbz300+s1oSUstaOfmaLw0+Dfo=,225256,465,Rihanna,Mikkel S. Eriksen| Sia Furler| B Levin| T.E. H...,NaN,52.0,1.000000
1,u9z+kvlYCkrSlDdJv3O/t5h+JpZJrSn2ZBI1GWe+EYQ=,202013,465,Miley Cyrus,Lukasz Gottwald| Claude Kelly| Jessica Cornish,NaN,52.0,0.860061
2,VFWmrWVR8IbIiP4iwS+RwATCVRqZozRMJ06PqQQeEBI=,215202,465,Rihanna,Calvin Harris,NaN,52.0,0.834578
3,4/f7S7iNOygpoPNxUzMv7jOPLg4HGYhpbEO7i41zUzU=,265142,465,Nelly Furtado,NaN,NaN,52.0,0.816846
4,J8poi1Mg9pKUq7Kw/w9y7JVmX3AH5OeFh4YJJi3hieY=,200341,465,Nicki Minaj,Nicki Minaj| D. Johnson| E. Dean,NaN,52.0,0.816453
5,FsI4pS2XGlzWGBGeSonZ8mvxsq3wHjRF19VjE2/elzc=,199320,465,Nicki Minaj|Jessie J|Ariana Grande,Rickard Goransson| Max Martin| Savan Kotecha| ...,NaN,52.0,0.786667
6,uuv/oI/dfOrHOHwDN9co6xyS484UCmel5TPfqOWCiU8=,296124,465,Rihanna,Alexander Grant| Skylar Grey| Marshall Mathers,NaN,52.0,0.782698
7,2AEXIjiK5meckRxl3k3zv5FnpimIvBrtgsI6FyFOlFI=,243461,465,Rihanna,Mikkel S. Eriksen| Tor Erik Hermansen| Sandy W...,NaN,52.0,0.779018
8,ES86JRfxcI9nxEfNv70Y772KQc9gxpykvAD7XRTsBgM=,284467,465,Will.I.Am,Jef Martens| William Adams| Jean Baptiste,NaN,52.0,0.778792
9,lhrN5Q069mAF35yPS+P4Rk1c1L4jhrAt3T2UjvT8yXc=,235415,465,Rihanna,Crystal Johnson| Mikkel S. Eriksen| Tor Erik H...,NaN,52.0,0.773167


Посмотрим на симилары. К Рианне советуется разнообразная попса (включая саму Рианну). Уже неплохо.

In [14]:
song_embeddings = pd.DataFrame(model[model.wv.vocab])

/usr/local/lib/python3.6/dist-packages/ipykernel_launcher.py:1: DeprecationWarning: Call to deprecated `__getitem__` (Method will be removed in 4.0.0, use self.wv.__getitem__() instead).
  """Entry point for launching an IPython kernel.


In [15]:
song_embeddings['song_id'] = np.array(list(model.wv.vocab.keys()))

In [ ]:
# song_embeddings.to_csv('/content/drive/My Drive/recsys-hw2/song_embeddings.csv')

In [16]:
song_embeddings.set_index('song_id', inplace=True)

song_embeddings.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
song_id,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,,
aRNy4rKgWyDwTjsMIcAbtpGvddDZ7L7OGUeJ09LAM2w=,-0.065482,-0.068227,-0.076048,-0.061993,-0.275134,0.171043,0.143375,-0.064375,0.054377,-0.036499,0.302783,-0.008307,-0.013507,-0.027456,-0.053242,-0.072875,0.115966,0.098605,0.025894,0.033993,0.045045,0.137854,-0.211253,-0.021682,0.165849,-0.022877,-0.062444,0.263192,-0.124918,-0.078338,-0.058289,0.342849,0.009679,-0.056216,0.078006,0.139037,0.100810,0.008379,0.029577,0.089867,-0.136818,0.122660,0.105650,-0.234720,0.016800,0.052276,-0.307390,-0.125612,-0.014116,-0.161481,0.049861,-0.052641,-0.087355,-0.072933,0.021762,-0.008092,0.089908,-0.013532,-0.139324,0.081746,-0.162995,0.095703,-0.166313,0.087881
J4qKkLIoW7aYACuTupHLAPZYmRp08en1AEux+GSUzdw=,0.123072,0.121616,0.070735,0.055963,-0.197980,0.089606,0.355758,-0.032625,-0.050772,0.037164,0.241058,-0.089028,0.015936,-0.010186,-0.045731,-0.104833,0.155668,0.050456,-0.145187,-0.023657,0.015698,0.188378,-0.150295,-0.014961,0.280791,0.003401,-0.080802,0.127153,-0.178997,0.034766,0.004712,0.140067,0.029839,-0.007055,0.046422,0.060884,0.089314,0.183581,-0.015595,0.029077,-0.040186,0.020245,0.117399,-0.139296,0.145882,-0.072066,-0.119903,-0.192283,0.064798,-0.156031,0.038094,0.043067,-0.085710,-0.104386,-0.038065,-0.020878,-0.116869,-0.080639,-0.135874,0.097824,-0.213864,-0.109046,-0.345735,0.047978
zHqZ07gn+YvF36FWzv9+y8KiCMhYhdAUS+vSIKY3UZY=,0.137898,0.010837,0.082906,0.062683,-0.232229,0.080065,0.384182,-0.036629,0.004754,0.016648,0.246122,-0.117699,0.021628,-0.042703,-0.028677,-0.149699,0.114986,0.058450,-0.078883,0.000012,0.020780,0.167563,-0.185695,-0.009151,0.311349,0.007549,-0.073311,0.222848,-0.121833,-0.000989,0.010010,0.163860,0.013469,0.034663,0.073978,0.061310,0.082892,0.061298,-0.011565,0.048590,-0.068041,0.047992,0.085246,-0.153258,0.209890,-0.043604,-0.100615,-0.135679,0.091745,-0.150705,0.058371,-0.016822,-0.186109,-0.068758,-0.078896,-0.023577,-0.131076,-0.070343,-0.163402,0.084853,-0.174468,-0.029823,-0.273987,0.038334
v/3onppBGoSpGsWb8iaCIO8eX5+iacbH5a4ZUhT7N54=,0.108986,0.010388,0.035941,-0.081749,-0.125901,0.018049,0.302114,-0.056385,0.077252,0.015209,0.156673,-0.071680,0.016839,-0.094711,0.048728,-0.139780,0.106519,0.060708,-0.137160,-0.001148,0.007079,0.235040,-0.107918,-0.055606,0.235175,0.101914,-0.089218,0.207804,-0.134677,-0.004845,0.071337,0.195270,-0.123330,0.053030,0.091173,0.031473,0.134627,0.124208,0.074474,-0.026325,-0.073629,0.026910,0.138178,-0.068511,0.201916,-0.190453,-0.169635,-0.133456,0.045682,-0.267005,0.058238,0.179948,-0.150425,-0.101763,-0.022035,-0.024360,-0.153630,-0.138495,-0.100049,0.106726,-0.173090,0.041727,-0.231507,-0.083026
upH6pOAUd+iV/MkpzeELvqEFoTEIVhsV9eML8N7/gUM=,-0.006570,0.049689,0.030527,0.121336,-0.247545,0.052728,0.184778,0.020844,0.098079,0.314190,0.090424,-0.094622,0.046622,-0.204528,0.162175,0.107429,-0.026518,0.076588,-0.048932,0.082733,0.136035,0.244638,-0.034847,-0.033871,0.136856,0.072246,-0.231133,-0.006347,0.126674,-0.144275,-0.078575,0.022407,-0.039057,-0.103288,0.007195,0.253297,0.035888,0.146733,-0.169174,0.117912,0.024040,0.027711,0.130036,-0.257617,0.160199,0.080076,-0.076759,-0.151288,-0.046636,-0.202482,-0.106397,0.113211,-0.076922,-0.124776,0.107211,-0.021816,-0.096669,0.058710,0.003050,0.180500,-0.062451,0.060071,-0.120611,0.054631


Посчитаем эмбеддинг юзера как среднее арифметическое эмбеддингов послушанных им песен.

In [17]:
%%time

from collections import defaultdict

def get_user_embeddings(df):
    user_embeddings = defaultdict(lambda: np.zeros(64))
    user_counts = defaultdict(int)

    for (_, row) in df.iterrows():
        user_id = row['msno']
        song_id = row['song_id']

        if song_id in model.wv:
            user_embeddings[user_id] += model.wv[song_id]
            user_counts[user_id] += 1

    for user_id in user_embeddings.keys():
        user_embeddings[user_id] /= user_counts[user_id]

    return user_embeddings

user_embeddings = get_user_embeddings(train)

CPU times: user 5min 56s, sys: 4.07 s, total: 6min
Wall time: 5min 54s


In [18]:
user_embeddings = pd.DataFrame(user_embeddings)
user_embeddings = user_embeddings.T

user_embeddings.head(5)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63
FGtllVqz18RPiwJj/edr2gV78zirAiY/9SmYvia+kCg=,0.069901,0.073953,-0.032212,0.000472,-0.118343,0.016907,0.109984,0.039110,0.086311,0.024692,0.019154,-0.087586,0.087357,-0.107056,-0.031364,-0.032746,0.026393,0.071422,0.015468,0.065759,0.045883,0.132439,-0.033470,0.028301,0.204895,-0.075828,-0.100630,-0.017229,-0.121269,-0.104148,-0.012646,0.107071,0.004459,0.082482,0.118789,0.082648,0.060240,0.175069,-0.028988,-0.032095,0.019287,-0.017321,0.069408,-0.130342,0.055803,0.078873,-0.099231,-0.092024,-0.028302,-0.116559,0.017708,0.022267,-0.119736,-0.071081,0.088264,0.030368,-0.029415,-0.066534,-0.041380,0.009204,-0.005058,0.030885,-0.166631,0.065408
Xumu+NIjS6QYVxDS4/t3SawvJ7viT9hPKXmf0RtLNx8=,0.070406,0.074113,-0.031380,-0.044474,-0.107201,-0.022089,0.079737,-0.030974,0.111834,0.068444,-0.053600,-0.013654,0.055653,-0.014300,-0.095494,0.055245,0.058839,0.141558,-0.023482,-0.008233,0.091056,0.181059,0.028395,0.070091,0.127997,-0.004152,-0.126307,0.012609,-0.119321,-0.042544,-0.060841,0.101074,0.023788,-0.000279,0.087973,0.070452,0.072242,0.197478,-0.029754,-0.038434,-0.005093,-0.085228,0.133729,-0.150563,0.121651,0.087830,-0.104910,-0.024200,0.031306,-0.086116,-0.026115,0.002752,-0.134994,-0.058177,0.005983,-0.051952,-0.042451,-0.072453,-0.142436,0.045933,-0.038925,-0.020155,-0.182501,0.040430
uHqAtShXTRXju5GE8ri3ITsVFepPf8jUoCF7ffNOuqE=,0.084202,0.082645,-0.001542,0.007503,-0.100469,-0.012493,0.190441,-0.003231,0.079402,0.028022,0.074953,-0.055099,0.046237,-0.074504,-0.028408,-0.027804,0.037632,0.054984,0.020488,0.064814,0.072179,0.124876,-0.000928,0.069117,0.113603,0.010553,-0.174369,-0.008391,-0.076284,-0.085760,-0.058035,0.066130,-0.044742,0.008805,0.128376,0.100459,0.030774,0.157822,-0.030449,-0.002527,-0.027992,0.024661,0.105305,-0.156791,0.119982,0.126922,-0.055646,-0.020474,-0.020988,-0.130125,0.015659,-0.001907,-0.132311,-0.089469,0.064619,0.013247,-0.040705,-0.005446,-0.072414,0.042481,-0.052934,0.057516,-0.210013,0.012250
3g0bC24RD7QUeALY1IF/e8L6X3fHG298ZFIsCEG9T9Y=,0.136235,0.089652,-0.031988,-0.025421,-0.101274,0.000280,0.196004,-0.009738,0.102429,0.052062,0.020159,-0.028922,0.065176,-0.073217,-0.080490,-0.015032,0.052887,0.102377,-0.003287,-0.019338,0.071975,0.138398,0.017124,0.034581,0.150102,0.003370,-0.090949,-0.002076,-0.099420,-0.047392,-0.046819,0.093468,-0.002177,0.006186,0.112071,0.072598,0.055716,0.159024,-0.018136,-0.009414,0.000027,-0.015765,0.092385,-0.169830,0.099424,0.088029,-0.117882,-0.059311,0.021522,-0.074245,0.026322,0.052730,-0.155724,-0.076760,-0.001934,0.002480,-0.048896,-0.037503,-0.072855,0.052353,-0.096877,0.046654,-0.237322,0.029513
TJU0Gfvy7FB+r89bWovPKXTjuApTCiv3xg/tt5shR78=,0.099432,0.083866,-0.041204,-0.012974,-0.125529,-0.017112,0.157816,-0.033481,0.089802,0.031595,0.003829,-0.022582,0.045221,-0.076903,-0.104897,0.000840,0.059522,0.122442,-0.027000,0.017449,0.070512,0.165256,0.002946,0.088069,0.141967,-0.003427,-0.136207,0.012283,-0.095177,-0.048772,-0.019894,0.088041,0.007355,0.005807,0.079717,0.094132,0.062962,0.178599,-0.037797,-0.037711,-0.035017,-0.046701,0.145730,-0.161921,0.100023,0.074327,-0.115608,-0.062065,0.039044,-0.105949,0.001854,0.013195,-0.145504,-0.067884,-0.009393,0.000158,-0.044369,-0.054302,-0.104477,0.054222,-0.052817,0.011744,-0.219931,0.021507


In [ ]:
# res.to_csv('/content/drive/My Drive/recsys-hw2/user_embeddings.csv')

Сохранил эмбеддинги в .csv, чтобы использовать их в ноутбуке с градиентным бустингом.

## Посчитаем AUC

In [20]:
train = pd.read_csv('/content/drive/My Drive/recsys-hw2/train.csv')

train.drop(['source_system_tab', 'source_screen_name', 'source_type'], axis=1, inplace=True)

target = train['target']

train.drop(['target'], axis=1, inplace=True)

In [21]:
MAX_SIZE = 1000000

train = train[:MAX_SIZE]
target = target[:MAX_SIZE]

In [22]:
def get_dot_product(song_id, user_id):
  if song_id not in song_embeddings.index:
    return 0
  if user_id not in user_embeddings.index:
    return 0
  
  return (np.sum(user_embeddings.loc[user_id] * song_embeddings.loc[song_id]))

In [24]:
%%time

train['dot_product'] = train.apply(lambda row: get_dot_product(row['song_id'], row['msno']), axis = 1)

CPU times: user 8min 56s, sys: 10.3 s, total: 9min 6s
Wall time: 8min 50s


In [27]:
from sklearn.metrics import roc_auc_score

score = roc_auc_score(target, train['dot_product'])

print(score)

0.6686055479653281


Ожидаемо, roc_auc = 0.66 оказался слабее чем у простого катбуста (0.75) и катбуста с эмбеддингами (0.86)